In [1]:
import pickle, time, os
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn
from tqdm import tqdm

from visualdl import LogWriter
logwriter = LogWriter(logdir='./runs')
# visualdl --logdir ./runs/ --host 0.0.0.0 --port 8040

users_df = pd.read_csv('data/csv/users.csv')
items_df = pd.read_csv('data/csv/items.csv')

In [2]:
emb_scale = 64
batch_size = 8192
len_users = len(users_df)
len_items = len(items_df)

In [23]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.data = data
        self.w = len(data[0])
        self.h = len(data)
        pass
    def __getitem__(self, idx):
        w = idx % self.w
        h = int(idx / self.h)
        n = np.array([w, h])
        return n, self.data[w][h]
    def __len__(self):
        return self.w * self.h
    pass 

In [26]:
data

array([[ 2234, 23222,     0],
       [ 2300,   207,     0],
       [ 2325, 20904,     0],
       ...,
       [ 2263, 11815,     0],
       [ 2228, 10733,     0],
       [ 2269,  2081,     0]])

In [25]:
file = open('data/net/train/17.pkl', 'rb')
data = pickle.load(file)
file.close()    
for x in Dataset(data):
    print(x)

(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0, 0]), 2234)
(array([1, 0]), 2300)
(array([2, 0]), 2325)
(array([0,

KeyboardInterrupt: 

In [11]:
class Net(nn.Layer):
    def __init__(self):
        super(Net,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.fc1 = nn.Linear(2 * emb_scale, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        pass
    def forward(self, input):
        user = self.users_emb(input[:,0])
        item = self.items_emb(input[:,1])
        x = paddle.concat([user, item], axis=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    pass
net = Net()
optim = paddle.optimizer.Adam(parameters=net.parameters(), learning_rate=0.005)
m = paddle.metric.Recall()

In [14]:
train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch_id in range(10):

    train_loss, train_acc, eval_loss, eval_acc = [], [], [], []

    file_list = os.listdir('data/net/train/')
    file = open('data/net/train/'+np.random.choice(file_list), 'rb')
    train_dataset = paddle.io.DataLoader(Dataset(pickle.load(file)),
                                         drop_last=True,
                                         batch_size=batch_size)
    file.close()    

    net.train()
    m.reset()
    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = net(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        m.update(y_pred, y_data)
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss.append(loss.numpy())
        train_acc.append(acc.numpy())

        logwriter.add_scalar("train_loss", value=loss.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("train_acc", value=acc.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("train_recall", value=m.accumulate(), step=batch_id+epoch_id*(batch_size))

    file_list = os.listdir('data/net/eval/')
    file = open('data/net/eval/'+np.random.choice(file_list), 'rb')
    train_dataset = paddle.io.DataLoader(Dataset(pickle.load(file)),
                                         drop_last=True,
                                         batch_size=batch_size,
                                         shuffle=True)
    file.close() 

    net.eval()
    m.reset()
    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = net(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        m.update(y_pred, y_data)

        logwriter.add_scalar("eval_loss", value=loss.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("eval_acc", value=acc.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("eval_recall", value=m.accumulate(), step=batch_id+epoch_id*(batch_size))

    print("epoch_id: {}, batch_id: {}, loss: {}, acc: {}, recall: {}".format(epoch_id, batch_id+1, loss.numpy(), acc.numpy(), m.accumulate()))

  3%|▎         | 17/629 [00:00<00:34, 17.70it/s]


KeyboardInterrupt: 

In [15]:
data

[Tensor(shape=[8192, 2], dtype=int32, place=Place(cpu), stop_gradient=True,
        [[1, 0],
         [2, 0],
         [0, 0],
         ...,
         [0, 0],
         [1, 0],
         [2, 0]]),
 Tensor(shape=[8192], dtype=int32, place=Place(cpu), stop_gradient=True,
        [594, 598, 643, ..., 643, 594, 598])]

In [19]:
np.max(data[1].numpy())

643

In [27]:
data[1]

Tensor(shape=[8192], dtype=int32, place=Place(cpu), stop_gradient=True,
       [10600, 0    , 0    , ..., 10600, 10569, 2244 ])